# Reverse Proxy Sidecar Pattern

https://www.magalix.com/blog/implemeting-a-reverse-proxy-server-in-kubernetes-using-the-sidecar-pattern

In [2]:
%env k=microk8s.kubectl

env: k=microk8s.kubectl


In [4]:
!$k status

Error: unknown command "status" for "kubectl"
Run 'kubectl --help' for usage.


In [5]:
! microk8s.status --wait-ready

microk8s is running
addons:
cilium: disabled
dashboard: disabled
dns: disabled
fluentd: disabled
gpu: disabled
helm: disabled
ingress: disabled
istio: disabled
jaeger: disabled
juju: disabled
knative: disabled
kubeflow: disabled
linkerd: disabled
metallb: disabled
metrics-server: disabled
prometheus: disabled
rbac: disabled
registry: disabled
storage: disabled


In [6]:
! make run

make[1]: Entering directory '/opt/k8s101/00-notebooks/06-day-6'
docker run -d -p 5000:5000 tbutzer/flasksidecar
f461e6764a539e7e01c76ab5d96a06763651e2e7c385cbab71993c3aea96f18d
docker: Error response from daemon: driver failed programming external connectivity on endpoint condescending_torvalds (2aa1cabec2e685681c6ced31f0132457cb2ca51c228f3925027f23a70b3dcee7): Bind for 0.0.0.0:5000 failed: port is already allocated.
Makefile:9: recipe for target 'run' failed
make[1]: *** [run] Error 125
make[1]: Leaving directory '/opt/k8s101/00-notebooks/06-day-6'


In [7]:
! curl localhost:5000

{"message":"Hello World!"}


In [8]:
! docker ps

CONTAINER ID        IMAGE                  COMMAND                  CREATED             STATUS              PORTS                    NAMES
9a15cb549d7b        tbutzer/flasksidecar   "gunicorn --bind 0.0…"   7 minutes ago       Up 7 minutes        0.0.0.0:5000->5000/tcp   pedantic_sanderson


In [9]:
! docker kill 9a15cb549d7b
! docker rm 9a15cb549d7b

9a15cb549d7b
9a15cb549d7b


In [13]:
# deployment and service
! $k apply -f g_deployment.yaml

service/hello-svc created
deployment.apps/hello-deployment created


In [14]:
! $k get pods -o wide

NAME                                READY   STATUS              RESTARTS   AGE   IP       NODE   NOMINATED NODE   READINESS GATES
hello-deployment-59fb4b89c9-4mbxf   0/1     ContainerCreating   0          14s   <none>   k8s    <none>           <none>
hello-deployment-59fb4b89c9-wmvrv   0/1     ContainerCreating   0          14s   <none>   k8s    <none>           <none>


In [15]:
! $k get svc

NAME         TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)        AGE
hello-svc    NodePort    10.152.183.168   <none>        80:32001/TCP   2m8s
kubernetes   ClusterIP   10.152.183.1     <none>        443/TCP        143m


In [16]:
! curl localhost:32001

{"message":"Hello World!"}


In [17]:
# deployment and service
! $k apply -f full_deployment.yaml

service/hello-svc configured
deployment.apps/hello-deployment configured


In [18]:
! $k get svc

NAME         TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)        AGE
hello-svc    NodePort    10.152.183.168   <none>        80:32001/TCP   12m
kubernetes   ClusterIP   10.152.183.1     <none>        443/TCP        153m


In [19]:
! $k get pods

NAME                                READY   STATUS    RESTARTS   AGE
hello-deployment-6b9454d7f5-gwsbg   2/2     Running   0          34s
hello-deployment-6b9454d7f5-hf5lk   2/2     Running   0          27s


In [20]:
! curl localhost:32001

<!DOCTYPE html>
<html>

<head>
   <meta charset="utf-8">
   <title>Hello World</title>
</head>

<body>
   <h1&Message from backend API</h1>
   <div id="backend"></div>
   <script src="http://code.jquery.com/jquery-1.9.1.min.js"></script>
   <script src="script.js"></script>
</body>

</html>


In [24]:
! curl localhost:32001/api/

{"message":"Hello World!"}


In [28]:
# deployment and service
! $k rollout -h

Manage the rollout of a resource.
  
 Valid resource types include:

  *  deployments
  *  daemonsets
  *  statefulsets

Examples:
  # Rollback to the previous deployment
  kubectl rollout undo deployment/abc
  
  # Check the rollout status of a daemonset
  kubectl rollout status daemonset/foo

Available Commands:
  history     View rollout history
  pause       Mark the provided resource as paused
  restart     Restart a resource
  resume      Resume a paused resource
  status      Show the status of the rollout
  undo        Undo a previous rollout

Usage:
  kubectl rollout SUBCOMMAND [options]

Use "kubectl <command> --help" for more information about a given command.
Use "kubectl options" for a list of global command-line options (applies to all
commands).


In [29]:
! $k get deployments

NAME               READY   UP-TO-DATE   AVAILABLE   AGE
hello-deployment   2/2     2            2           33m


In [32]:
!$k rollout restart deployment/hello-deployment

deployment.apps/hello-deployment restarted


In [33]:
! $k rollout undo deployment/hello-deployment

deployment.apps/hello-deployment rolled back


In [37]:
! $k get deploy


NAME               READY   UP-TO-DATE   AVAILABLE   AGE
hello-deployment   2/2     2            2           36m


In [38]:
!$k delete deploy hello-deployment

deployment.apps "hello-deployment" deleted


In [39]:
! $k get deploy

No resources found in default namespace.


In [40]:
! $k apply -f full_deployment.yaml

service/hello-svc unchanged
deployment.apps/hello-deployment created
